In [3]:
#GRPH
def parse_fasta(file_path):
    """Parse FASTA format data from a file and return a dictionary of label to DNA strings."""
    sequences = {}
    label = None
    with open(file_path, "r") as file:
        for line in file:
            if line.startswith(">"):
                label = line[1:].strip()  # Extract label without ">"
                sequences[label] = ""
            else:
                sequences[label] += line.strip()  # Append DNA sequence
    return sequences

def overlap_graph(sequences, k):
    """Generate the adjacency list of the overlap graph for k-length overlaps."""
    adjacency_list = []
    labels = list(sequences.keys())
    for i, label_s in enumerate(labels):
        suffix = sequences[label_s][-k:]  # Get suffix of length k
        for j, label_t in enumerate(labels):
            if i != j:  # Ensure s ≠ t
                prefix = sequences[label_t][:k]  # Get prefix of length k
                if suffix == prefix:
                    adjacency_list.append((label_s, label_t))
    return adjacency_list

# Main function to process input file and print adjacency list
def main():
    input_file = "INPUT.TXT"  # Replace with your input file name
    k = 3  # Length of overlap
    sequences = parse_fasta(input_file)
    adjacency_list = overlap_graph(sequences, k)
    for edge in adjacency_list:
        print(edge[0], edge[1])

if __name__ == "__main__":
    main()


Rosalind_0677 Rosalind_6341
Rosalind_0677 Rosalind_4518
Rosalind_5465 Rosalind_6355
Rosalind_3012 Rosalind_3372
Rosalind_3012 Rosalind_9672
Rosalind_0214 Rosalind_1640
Rosalind_0214 Rosalind_3724
Rosalind_5312 Rosalind_9793
Rosalind_7301 Rosalind_0835
Rosalind_7301 Rosalind_6401
Rosalind_2960 Rosalind_4309
Rosalind_2960 Rosalind_3350
Rosalind_2960 Rosalind_7998
Rosalind_2960 Rosalind_1672
Rosalind_9665 Rosalind_9128
Rosalind_1640 Rosalind_4735
Rosalind_3372 Rosalind_5388
Rosalind_3372 Rosalind_4772
Rosalind_8492 Rosalind_3524
Rosalind_3871 Rosalind_9793
Rosalind_6341 Rosalind_5289
Rosalind_6985 Rosalind_7458
Rosalind_5551 Rosalind_2780
Rosalind_7458 Rosalind_5289
Rosalind_6407 Rosalind_0214
Rosalind_6407 Rosalind_9103
Rosalind_6407 Rosalind_7879
Rosalind_8252 Rosalind_4309
Rosalind_8252 Rosalind_3350
Rosalind_8252 Rosalind_7998
Rosalind_8252 Rosalind_1672
Rosalind_9762 Rosalind_9972
Rosalind_9762 Rosalind_0045
Rosalind_6122 Rosalind_7043
Rosalind_8540 Rosalind_1640
Rosalind_8540 Rosali

In [53]:
#CTE 

import heapq
from collections import defaultdict

def dijkstra(n, graph, start, target):
    """Returns the shortest path from start to target using Dijkstra's algorithm."""
    dist = {i: float('inf') for i in range(1, n + 1)}
    dist[start] = 0
    pq = [(0, start)]  # (distance, node)
    
    while pq:
        d, node = heapq.heappop(pq)
        
        if d > dist[node]:
            continue
        
        for neighbor, weight in graph[node]:
            if dist[node] + weight < dist[neighbor]:
                dist[neighbor] = dist[node] + weight
                heapq.heappush(pq, (dist[neighbor], neighbor))
    
    return dist[target] if dist[target] != float('inf') else -1

def find_shortest_cycle(n, edges):
    """Find the shortest cycle passing through the first edge."""
    # Create the graph as an adjacency list
    graph = defaultdict(list)
    for u, v, w in edges:
        graph[u].append((v, w))
    
    # Consider the first edge
    u, v, w = edges[0]
    
    # Temporarily remove the edge (u, v)
    graph[u] = [edge for edge in graph[u] if edge[0] != v]
    
    # Run Dijkstra from v to u
    cycle_length = dijkstra(n, graph, v, u)
    
    # If a cycle is found, return the cycle length
    if cycle_length != -1:
        return cycle_length + w  # Add the weight of the edge (u, v)
    
    return -1

def read_graph_from_file(filename):
    """Reads multiple graphs from the input file."""
    graphs = []
    
    try:
        with open(filename, 'r') as file:
            k = int(file.readline().strip())  # Number of graphs
            
            for graph_idx in range(k):
                n, m = map(int, file.readline().split())  # Number of vertices and edges
                edges = []
                
                for _ in range(m):
                    u, v, w = map(int, file.readline().split())  # Read each edge
                    edges.append((u, v, w))
                
                graphs.append((n, edges))
    
    except Exception as e:
        print(f"Error reading file: {e}")
    
    return graphs

def main():
    input_file = "INPUT.TXT"  # Replace with your input file name
    graphs = read_graph_from_file(input_file)  # Read graphs from the file
    results = []

    # For each graph, find the shortest cycle through the first edge
    for n, edges in graphs:
        result = find_shortest_cycle(n, edges)
        results.append(result)

    # Output the results for each graph
    print(" ".join(map(str, results)))

if __name__ == "__main__":
    main()


-1 10


In [6]:
#TREE
from collections import defaultdict, deque

def parse_input(file_path):
    """Parse the input file to extract the number of nodes and edges."""
    with open(file_path, "r") as file:
        lines = file.read().strip().split("\n")
    
    n = int(lines[0])  # First line is the number of nodes
    edges = []
    for line in lines[1:]:
        u, v = map(int, line.split())
        edges.append((u, v))
    
    return n, edges

def find_connected_components(n, edges):
    """Find the number of connected components in the graph."""
    graph = defaultdict(list)
    
    # Build adjacency list
    for u, v in edges:
        graph[u].append(v)
        graph[v].append(u)
    
    visited = [False] * (n + 1)
    components = 0

    def bfs(start):
        """Perform BFS to visit all nodes in a component."""
        queue = deque([start])
        while queue:
            node = queue.popleft()
            for neighbor in graph[node]:
                if not visited[neighbor]:
                    visited[neighbor] = True
                    queue.append(neighbor)

    # Count connected components
    for node in range(1, n + 1):
        if not visited[node]:
            components += 1
            visited[node] = True
            bfs(node)

    return components

def minimum_edges_to_tree(n, edges):
    """Return the minimum number of edges to make the graph a tree."""
    components = find_connected_components(n, edges)
    # Minimum edges to connect all components into a tree
    return components - 1

def main():
    input_file = "INPUT.TXT"  # Replace with your input file name
    n, edges = parse_input(input_file)
    result = minimum_edges_to_tree(n, edges)
    print(result)

if __name__ == "__main__":
    main()


85


In [12]:
#LONG
def parse_fasta_file(file_path):
    """Parse a FASTA file and return a list of DNA strings."""
    sequences = []
    with open(file_path, "r") as file:
        current_sequence = ""
        for line in file:
            if line.startswith(">"):
                if current_sequence:
                    sequences.append(current_sequence)
                current_sequence = ""  # Start a new sequence
            else:
                current_sequence += line.strip()  # Append sequence data
        if current_sequence:
            sequences.append(current_sequence)  # Add the last sequence
    return sequences

def overlap(s1, s2, min_length=1):
    """Find the maximum overlap between the suffix of s1 and prefix of s2."""
    max_overlap = 0
    start = 0  # Start looking for overlaps of length `min_length`

    while True:
        start = s1.find(s2[:min_length], start)  # Look for prefix match
        if start == -1:  # No match found
            break
        # Check if the suffix of s1 starting at `start` matches the prefix of s2
        if s1[start:] == s2[:len(s1) - start]:
            overlap_len = len(s1) - start
            if overlap_len > max_overlap:
                max_overlap = overlap_len
        start += 1  # Move to the next possible match

    return max_overlap

def shortest_superstring(sequences):
    """Find the shortest superstring containing all sequences."""
    while len(sequences) > 1:
        max_overlap = -1
        best_pair = (0, 0)
        best_merged = ""

        # Find the pair of strings with the maximum overlap
        for i in range(len(sequences)):
            for j in range(len(sequences)):
                if i != j:
                    s1, s2 = sequences[i], sequences[j]
                    overlap_len = overlap(s1, s2, len(s1) // 2)
                    if overlap_len > max_overlap:
                        max_overlap = overlap_len
                        best_pair = (i, j)
                        best_merged = s1 + s2[overlap_len:]

        # Merge the best pair and remove the originals
        i, j = best_pair
        sequences[i] = best_merged
        del sequences[j]

    return sequences[0]

def main():
    input_file = "INPUT.TXT"  # Replace with your input file name
    sequences = parse_fasta_file(input_file)
    result = shortest_superstring(sequences)
    print(result)

if __name__ == "__main__":
    main()


...


In [18]:
#DEG
def read_graph(file_path):
    """Read the graph from a file and return the number of vertices and the edge list."""
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Parse the number of vertices and edges
    n, _ = map(int, lines[0].strip().split())
    
    # Parse the edge list
    edges = []
    for line in lines[1:]:
        line = line.strip()  # Remove leading/trailing spaces
        if not line:  # Skip blank lines
            continue
        parts = line.split()
        if len(parts) == 2:  # Check for valid edge definition
            try:
                u, v = map(int, parts)
                edges.append((u, v))
            except ValueError:
                # Skip lines that cannot be parsed into integers
                continue
    
    return n, edges

def compute_degrees(n, edges):
    """Compute the degree of each vertex in the graph."""
    # Initialize degree list with zeros
    degrees = [0] * n
    
    # Iterate through each edge
    for u, v in edges:
        degrees[u - 1] += 1  # Increment degree of vertex u
        degrees[v - 1] += 1  # Increment degree of vertex v (since graph is undirected)
    
    return degrees

def main():
    input_file = "INPUT.TXT"  # Replace with your input file name
    # Read graph data
    n, edges = read_graph(input_file)
    # Compute degrees
    degrees = compute_degrees(n, edges)
    # Print the result
    print(" ".join(map(str, degrees)))

if __name__ == "__main__":
    main()


15 16 26 16 26 19 22 20 14 18 23 21 19 21 29 20 16 16 18 19 19 20 23 23 24 25 20 24 20 20 16 20 11 20 18 16 18 24 19 14 22 17 17 27 14 15 15 18 24 21 22 18 16 29 17 23 15 28 18 23 23 22 21 27 30 20 20 21 19 13 15 19 16 23 16 24 26 20 19 18 20 19 18 18 22 17 14 23 16 19 24 16 20 18 15 21 18 21 16 22 19 18 37 16 19 17 21 25 13 21 15 16 29 27 12 21 16 15 16 18 22 21 18 31 25 26 27 18 10 13 25 23 17 23 14 21 28 29 19 17 19 16 23 27 16 17 19 22 15 14 18 14 16 23 19 25 20 19 12 25 21 19 17 17 25 24 25 27 18 17 21 22 17 18 22 21 25 22 21 12 21 16 23 19 26 15 22 19 28 22 14 11 17 25 19 25 15 17 10 22 25 22 22 17 21 18 22 20 20 19 16 21 18 23 20 15 23 26 20 20 11 14 22 21 19 19 18 22 22 27 19 15 26 21 21 23 28 27 27 16 20 28 17 30 31 21 17 20 13 19 26 18 25 17 20 21 23 26 15 19 16 21 15 16 17 18 18 20 19 24 13 18 21 26 12 19 16 12 28 24 18 25 14 17 18 23 18 18 19 17 21 20 20 20 17 19 16 17 22 24 17 16 22 16 21 15 25 17 18 21 20 23 23 16 26 16 18 14 19 14 29 24 20 23 23 22 19 21 21 23 22 28 20 1

In [21]:
#DDEG

def read_graph(file_path):
    """Read the graph from a file and return the number of vertices and the adjacency list."""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Parse the number of vertices and edges
    n, _ = map(int, lines[0].split())

    # Initialize the adjacency list
    adj_list = {i: [] for i in range(1, n + 1)}

    # Parse the edges
    for line in lines[1:]:
        if line.strip():  # Skip empty lines
            parts = line.split()
            if len(parts) == 2:  # Ensure the line has exactly two elements
                u, v = map(int, parts)
                adj_list[u].append(v)
                adj_list[v].append(u)  # Since the graph is undirected

    return n, adj_list


def compute_neighbor_degree_sums(n, adj_list):
    """Compute the sum of the degrees of the neighbors for each vertex."""
    # Initialize the degree sums array
    neighbor_degree_sums = [0] * n

    # Iterate through each vertex to compute neighbor degree sums
    for vertex in range(1, n + 1):
        neighbor_sum = 0
        for neighbor in adj_list[vertex]:
            # Degree of a neighbor is simply the length of its adjacency list
            neighbor_sum += len(adj_list[neighbor])
        # Store the sum of neighbor degrees for the current vertex
        neighbor_degree_sums[vertex - 1] = neighbor_sum

    return neighbor_degree_sums


def main():
    input_file = "INPUT.TXT"  # Replace with your input file name
    # Read graph data
    n, adj_list = read_graph(input_file)
    # Compute the sum of the degrees of the neighbors
    neighbor_degree_sums = compute_neighbor_degree_sums(n, adj_list)
    # Print the result
    print(" ".join(map(str, neighbor_degree_sums)))


if __name__ == "__main__":
    main()



303 284 535 375 341 245 524 313 500 331 272 375 294 483 384 413 462 494 308 445 376 356 589 346 360 456 623 447 372 328 275 340 387 341 262 376 329 531 570 275 346 432 498 457 433 585 397 437 477 348 389 372 382 549 376 380 533 699 466 474 405 430 372 250 440 634 295 543 490 545 464 386 395 480 405 453 361 351 143 350 521 408 364 289 399 404 283 365 365 361 524 308 466 366 476 344 393 455 435 409 454 677 366 395 528 564 377 499 447 258 326 434 449 435 377 394 421 288 485 406 386 430 444 486 434 379 574 674 579 414 204 356 439 573 463 451 417 433 345 550 521 380 298 346 404 378 438 413 422 295 339 389 370 390 460 232 487 186 483 434 604 452 542 308 520 392 492 367 416 216 358 466 312 474 413 316 347 250 226 289 375 446 395 419 545 433 275 449 225 321 430 277 423 576 363 580 278 379 219 453 329 281 420 403 439 425 264 488 423 468 591 425 428 308 341 387 403 449 457 318 512 538 453 404 285 533 550 261 328 395 369 378 443 330 555 603 463 654 388 261 453 467 381 518 290 438 308 428 321 394 

In [23]:
#CC

def read_graph(file_path):
    """Reads the graph from a file and returns the number of vertices and adjacency list."""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Parse number of vertices and edges
    n, _ = map(int, lines[0].split())

    # Initialize adjacency list
    adj_list = {i: [] for i in range(1, n + 1)}

    # Populate adjacency list
    for line in lines[1:]:
        if line.strip():
            u, v = map(int, line.split())
            adj_list[u].append(v)
            adj_list[v].append(u)  # Since the graph is undirected

    return n, adj_list


def dfs(node, adj_list, visited):
    """Perform DFS from the given node, marking all reachable nodes as visited."""
    stack = [node]
    while stack:
        current = stack.pop()
        if not visited[current]:
            visited[current] = True
            for neighbor in adj_list[current]:
                if not visited[neighbor]:
                    stack.append(neighbor)


def count_connected_components(n, adj_list):
    """Counts the number of connected components in the graph."""
    visited = {i: False for i in range(1, n + 1)}
    connected_components = 0

    for node in range(1, n + 1):
        if not visited[node]:  # New component found
            dfs(node, adj_list, visited)
            connected_components += 1

    return connected_components


def main():
    input_file = "INPUT.TXT"  # Replace with your input file
    # Read graph data
    n, adj_list = read_graph(input_file)
    # Count connected components
    result = count_connected_components(n, adj_list)
    print(result)


if __name__ == "__main__":
    main()


116


In [25]:
#BFS

from collections import deque

def read_graph(file_path):
    """Reads the graph from a file and returns the number of vertices and adjacency list."""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Parse number of vertices and edges
    n, _ = map(int, lines[0].split())

    # Initialize adjacency list
    adj_list = {i: [] for i in range(1, n + 1)}

    # Populate adjacency list
    for line in lines[1:]:
        if line.strip():
            u, v = map(int, line.split())
            adj_list[u].append(v)

    return n, adj_list

def bfs(n, adj_list):
    """Compute the single-source shortest distances from vertex 1."""
    # Initialize distances array with -1 (indicating unvisited)
    distances = [-1] * n
    distances[0] = 0  # Distance to vertex 1 is 0
    
    # Queue for BFS
    queue = deque([1])  # Start BFS from vertex 1
    
    while queue:
        current = queue.popleft()
        
        # Visit all neighbors of the current node
        for neighbor in adj_list[current]:
            if distances[neighbor - 1] == -1:  # If not visited
                distances[neighbor - 1] = distances[current - 1] + 1
                queue.append(neighbor)
    
    return distances

def main():
    input_file = "INPUT.TXT"  # Replace with your input file
    # Read graph data
    n, adj_list = read_graph(input_file)
    # Compute the shortest distances using BFS
    result = bfs(n, adj_list)
    # Print the result
    print(" ".join(map(str, result)))

if __name__ == "__main__":
    main()


0 6 -1 4 6 2 7 7 7 7 -1 6 5 5 -1 4 6 4 6 5 5 7 5 7 6 6 5 7 5 7 5 7 -1 6 7 7 6 5 4 6 7 4 3 6 8 5 5 7 6 5 7 6 -1 5 8 6 6 4 6 5 6 7 8 5 6 6 8 8 8 6 7 7 5 -1 5 4 6 4 6 6 6 6 5 3 5 4 6 5 7 7 4 8 5 6 5 5 4 6 7 6 5 6 5 6 6 9 7 5 5 5 6 6 6 5 7 6 6 7 6 6 4 7 6 7 6 3 5 6 5 6 6 8 5 -1 -1 8 6 6 7 7 5 7 6 8 5 4 4 7 8 4 6 6 6 6 2 4 6 7 6 -1 6 8 4 3 6 7 6 8 4 6 5 6 7 5 5 6 7 6 4 7 6 6 6 5 6 7 6 7 -1 6 6 6 4 -1 4 6 5 7 5 -1 5 5 5 7 4 4 6 6 5 5 6 8 7 6 4 5 5 6 6 7 5 6 5 7 4 6 6 6 7 6 6 5 6 8 6 7 10 5 7 7 6 6 8 6 6 7 6 6 5 6 6 7 7 6 7 5 9 3 7 6 7 7 8 7 6 4 5 6 6 6 3 7 6 6 3 7 7 6 5 6 8 -1 7 7 5 6 8 6 7 5 5 6 6 6 6 6 6 6 7 7 7 6 5 2 6 7 7 7 6 6 5 6 5 2 7 6 7 -1 5 6 7 5 6 7 5 6 5 4 6 6 6 5 6 7 7 5 7 7 6 6 3 5 4 6 5 7 5 7 7 4 6 4 5 5 6 5 5 5 6 7 7 7 7 6 7 7 6 7 7 4 7 7 5 5 6 5 6 4 6 1 7 5 6 6 7 -1 4 5 5 4 4 5 8 5 7 4 5 7 6 6 6 6 7 5 6 5 7 5 7 6 6 6 7 6 5 8 4 6 8 5 1 6 5 7 7 -1 6 6 6 8 6 -1 5 5 6 8 5 7 5 5 6 6 6 6 8 5 6 5 6 5 7 6 6 5 4 6 7 6 6 7 5 2 6 5 5 1 6 4 6 7 6 3 -1 6 3 6 -1 5 6 7 6 6 6 5 2 4 6 6 1 6 

In [27]:
#DIJ

import heapq

def read_graph(file_path):
    """Reads the graph from a file and returns the number of vertices and adjacency list."""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Parse number of vertices and edges
    n, _ = map(int, lines[0].split())

    # Initialize adjacency list
    adj_list = {i: [] for i in range(1, n + 1)}

    # Populate adjacency list with edges and weights
    for line in lines[1:]:
        if line.strip():
            u, v, w = map(int, line.split())
            adj_list[u].append((v, w))  # directed edge u -> v with weight w

    return n, adj_list

def dijkstra(n, adj_list):
    """Computes the shortest distances from vertex 1 to all other vertices using Dijkstra's algorithm."""
    # Initialize distances with infinity
    distances = [float('inf')] * n
    distances[0] = 0  # Distance to the source (vertex 1) is 0
    
    # Min-heap for priority queue (distance, vertex)
    priority_queue = [(0, 1)]  # Start from vertex 1 with distance 0
    
    while priority_queue:
        current_distance, current_vertex = heapq.heappop(priority_queue)
        
        # If this distance is greater than the already found distance, skip it
        if current_distance > distances[current_vertex - 1]:
            continue
        
        # Explore neighbors of the current vertex
        for neighbor, weight in adj_list[current_vertex]:
            distance = current_distance + weight
            # If a shorter path to neighbor is found, update the distance
            if distance < distances[neighbor - 1]:
                distances[neighbor - 1] = distance
                heapq.heappush(priority_queue, (distance, neighbor))
    
    # Convert unreachable distances to -1
    for i in range(n):
        if distances[i] == float('inf'):
            distances[i] = -1
    
    return distances

def main():
    input_file = "INPUT.TXT"  # Replace with your input file
    # Read graph data
    n, adj_list = read_graph(input_file)
    # Compute the shortest distances using Dijkstra's algorithm
    result = dijkstra(n, adj_list)
    # Print the result
    print(" ".join(map(str, result)))

if __name__ == "__main__":
    main()


0 2282 3246 -1 3033 2452 1763 2215 2745 2840 3012 2683 2579 2430 2096 2205 2516 -1 2420 3383 3891 2127 3208 2473 1970 2900 2454 1161 2329 3015 1650 2706 1492 1653 1850 4133 4 344 2551 1768 1722 -1 3366 2883 3533 2084 2805 2737 2169 -1 -1 2897 2572 1821 3826 1703 3740 2707 2854 2699 2876 2638 1859 3008 1589 2250 -1 2936 2607 2173 2296 2817 2324 -1 1471 1454 1029 2292 2609 2501 1921 2614 2233 2593 3098 512 3109 2510 2095 2386 2184 2715 2624 1783 2533 3365 1960 2971 2341 2670 1692 2004 1556 1265 1227 2581 2634 2370 2156 1799 1605 3116 1894 -1 2412 4647 1607 2422 2810 1541 1956 2614 -1 2362 2493 3399 1435 3004 2071 2564 3153 2714 1886 2094 1573 2151 2048 2383 2305 2903 2823 -1 2448 2472 -1 1377 3648 1613 1658 938 2231 3392 1939 3174 1999 2075 2419 1794 1621 2748 3157 2054 2338 3665 2370 2092 1296 1655 1640 2439 2026 1853 3113 1779 1919 2102 3711 2362 2045 2278 2612 2192 2326 3013 2601 2197 2240 4238 2110 2866 -1 2105 889 1776 1967 1518 -1 3099 2327 3004 1479 2580 2364 2615 3418 1796 2333 2

In [73]:
#DAG
def dfs(v, adj_list, visited, rec_stack):
    """Performs DFS and detects cycle."""
    # If node is being visited in the current DFS path
    if rec_stack[v] == 1:
        return True
    # If node is already processed, no cycle from this node
    if visited[v] == 2:
        return False

    # Mark the current node as visited and part of recursion stack
    visited[v] = 1
    rec_stack[v] = 1

    # Recur for all the vertices adjacent to this vertex
    for neighbor in adj_list[v]:
        if dfs(neighbor, adj_list, visited, rec_stack):
            return True

    # Mark the current node as fully processed
    visited[v] = 2
    rec_stack[v] = 0
    return False

def is_acyclic(n, edges):
    """Check if the directed graph with n vertices and given edges is acyclic."""
    adj_list = {i: [] for i in range(1, n + 1)}
    
    # Build the adjacency list
    for u, v in edges:
        adj_list[u].append(v)

    # Initialize visited and recursion stack arrays
    visited = [0] * (n + 1)  # 0: unvisited, 1: visiting, 2: visited
    rec_stack = [0] * (n + 1)  # Recursion stack to detect cycles

    # Run DFS for each node
    for node in range(1, n + 1):
        if visited[node] == 0:
            if dfs(node, adj_list, visited, rec_stack):
                return -1  # A cycle detected

    return 1  # No cycle detected

def read_graph_from_file(filename):
    """Reads a graph from the input file and returns the graph details."""
    graphs = []
    
    try:
        with open(filename, 'r') as file:
            k = int(file.readline().strip())  # Number of graphs
            print(f"Reading {k} graphs")
            
            for graph_idx in range(k):
                # Read the number of vertices and edges for each graph
                line = file.readline().strip()
                if not line:  # Handle any empty lines
                    print(f"Skipping empty line at the start of graph {graph_idx + 1}")
                    continue
                    
                n, m = map(int, line.split())  # Number of vertices and edges
                edges = []
                
                print(f"Reading {m} edges for graph {graph_idx + 1} with {n} vertices")
                
                for _ in range(m):
                    edge_line = file.readline().strip()
                    if not edge_line:  # Handle empty lines
                        print("Skipping empty edge line")
                        continue
                    
                    u, v = map(int, edge_line.split())  # Read each edge
                    edges.append((u, v))
                
                graphs.append((n, edges))
                
    except Exception as e:
        print(f"Error reading file: {e}")
    
    return graphs

def main():
    input_file = "INPUT.TXT"  # Replace with your input file name
    graphs = read_graph_from_file(input_file)  # Read graphs from the file
    results = []

    # Check if each graph is acyclic
    for n, edges in graphs:
        result = is_acyclic(n, edges)
        results.append(result)

    # Output the results for each graph
    print(" ".join(map(str, results)))

if __name__ == "__main__":
    main()



Reading 3 graphs
Reading 4 edges for graph 1 with 4 vertices
Reading 2 edges for graph 2 with 3 vertices
Reading 1 edges for graph 3 with 3 vertices
Skipping empty edge line
-1 1 1


In [75]:
#BF

def bellman_ford(n, edges):
    """Bellman-Ford algorithm to find single-source shortest paths in a directed graph."""
    INF = float('inf')
    dist = [INF] * (n + 1)  # Initialize distances to all vertices as infinity
    dist[1] = 0  # Distance to the source vertex is 0

    # Step 1: Relax edges (n - 1) times
    for _ in range(n - 1):
        for u, v, w in edges:
            if dist[u] != INF and dist[u] + w < dist[v]:
                dist[v] = dist[u] + w

    # Step 2: Check for negative weight cycles (not required here per the problem)
    for u, v, w in edges:
        if dist[u] != INF and dist[u] + w < dist[v]:
            raise ValueError("Graph contains a negative weight cycle")

    # Step 3: Replace unreachable nodes with 'x'
    result = []
    for i in range(1, n + 1):
        if dist[i] == INF:
            result.append('x')
        else:
            result.append(str(dist[i]))

    return result


def read_graph(input_file):
    """Reads the graph data from the input file."""
    with open(input_file, 'r') as file:
        first_line = file.readline().strip()
        n, m = map(int, first_line.split())  # Number of vertices and edges
        edges = []
        for _ in range(m):
            u, v, w = map(int, file.readline().strip().split())
            edges.append((u, v, w))
    return n, edges


def main():
    input_file = "INPUT.TXT"  # Replace with your input file
    n, edges = read_graph(input_file)  # Read the graph
    try:
        result = bellman_ford(n, edges)  # Run Bellman-Ford algorithm
        print(" ".join(result))  # Print the results
    except ValueError as e:
        print(str(e))


if __name__ == "__main__":
    main()


0 5 5 6 9 7 9 8 x


In [64]:
#NWC
def bellman_ford(n, edges):
    """Uses Bellman-Ford to detect negative weight cycles."""
    # Initialize distances from source (vertex 1)
    dist = [float('inf')] * (n + 1)
    dist[1] = 0  # Distance to source is 0

    # Relax all edges (n-1) times
    for _ in range(n - 1):
        for u, v, w in edges:
            if dist[u] != float('inf') and dist[u] + w < dist[v]:
                dist[v] = dist[u] + w

    # Check for negative weight cycle
    for u, v, w in edges:
        if dist[u] != float('inf') and dist[u] + w < dist[v]:
            return 1  # Negative weight cycle detected

    return -1  # No negative weight cycle

def read_graph_from_file(filename):
    """Reads graphs from a file and returns a list of graphs with vertices and edges."""
    graphs = []
    
    try:
        with open(filename, 'r') as file:
            k = int(file.readline().strip())  # Number of graphs
            
            for graph_idx in range(k):
                line = file.readline().strip()
                if not line:
                    continue  # Skip any empty lines

                n, m = map(int, line.split())  # Number of vertices and edges
                edges = []
                
                for _ in range(m):
                    edge_line = file.readline().strip()
                    if edge_line:
                        u, v, w = map(int, edge_line.split())
                        edges.append((u, v, w))
                
                graphs.append((n, edges))

    except Exception as e:
        print(f"Error reading file: {e}")
    
    return graphs

def main():
    input_file = "INPUT.TXT"  # Replace with your input file name
    graphs = read_graph_from_file(input_file)  # Read graphs from the file
    results = []

    # For each graph, check for negative weight cycle
    for n, edges in graphs:
        result = bellman_ford(n, edges)
        results.append(result)

    # Output the results for each graph
    print(" ".join(map(str, results)))

if __name__ == "__main__":
    main()


-1 1


In [69]:
#SDAG

from collections import defaultdict

def topological_sort(n, adj_list):
    """Performs topological sorting of the DAG using DFS."""
    visited = [False] * (n + 1)
    topo_order = []

    def dfs(v):
        visited[v] = True
        for neighbor, _ in adj_list[v]:  # Only use the vertex part of the tuple
            if not visited[neighbor]:
                dfs(neighbor)
        topo_order.append(v)
    
    for vertex in range(1, n + 1):
        if not visited[vertex]:
            dfs(vertex)
    
    # Return the reverse of the topo_order since we want vertices in finishing order
    return topo_order[::-1]

def shortest_paths_in_dag(n, edges):
    """Returns the shortest paths from vertex 1 to all other vertices in the DAG."""
    # Create adjacency list and initialize distance array
    adj_list = defaultdict(list)
    for u, v, weight in edges:
        adj_list[u].append((v, weight))  # Store the vertex and weight as a tuple
    
    dist = [float('inf')] * (n + 1)
    dist[1] = 0  # Distance to the source vertex is 0

    # Step 1: Topological sort the DAG
    topo_order = topological_sort(n, adj_list)
    
    # Step 2: Relax the edges in topologically sorted order
    for u in topo_order:
        if dist[u] != float('inf'):  # Only process nodes that are reachable
            for v, weight in adj_list[u]:  # Unpack the tuple
                if dist[v] > dist[u] + weight:
                    dist[v] = dist[u] + weight
    
    # Step 3: Handle unreachable vertices by setting them to 'x'
    result = []
    for i in range(1, n + 1):
        if dist[i] == float('inf'):
            result.append('x')
        else:
            result.append(str(dist[i]))
    
    return result

def read_graph_from_file(filename):
    """Reads a graph from the input file and returns the graph details."""
    with open(filename, 'r') as file:
        n, m = map(int, file.readline().split())
        edges = []
        
        for _ in range(m):
            u, v, w = map(int, file.readline().split())
            edges.append((u, v, w))
    
    return n, edges

def main():
    input_file = "INPUT.TXT"  # Replace with the actual input file name
    n, edges = read_graph_from_file(input_file)
    
    # Get the shortest paths from vertex 1 to all other vertices
    result = shortest_paths_in_dag(n, edges)
    
    # Output the result
    print(" ".join(result))

if __name__ == "__main__":
    main()




0 x -4 -2 -3


In [60]:
#BIP
from collections import deque

def is_bipartite(n, edges):
    """Check if the graph is bipartite using BFS."""
    # Create an adjacency list
    adj_list = {i: [] for i in range(1, n + 1)}
    for u, v in edges:
        adj_list[u].append(v)
        adj_list[v].append(u)  # Since the graph is undirected
    
    # Array to store the color of each vertex (0: uncolored, 1: color 1, -1: color 2)
    colors = [0] * (n + 1)  # We use 1-based indexing
    
    def bfs(start):
        """Perform BFS to check if graph is bipartite starting from node `start`."""
        queue = deque([start])
        colors[start] = 1  # Start coloring with color 1
        
        while queue:
            node = queue.popleft()
            current_color = colors[node]
            
            for neighbor in adj_list[node]:
                if colors[neighbor] == 0:  # Not colored, assign the opposite color
                    colors[neighbor] = -current_color
                    queue.append(neighbor)
                elif colors[neighbor] == current_color:  # Same color, conflict
                    return False
        return True
    
    # Check all components of the graph
    for node in range(1, n + 1):
        if colors[node] == 0:  # If the node hasn't been visited
            if not bfs(node):
                return -1  # If any component is not bipartite
    
    return 1  # All components are bipartite

def read_graph_from_file(filename):
    """Reads multiple graphs from the input file."""
    graphs = []
    
    try:
        with open(filename, 'r') as file:
            k = int(file.readline().strip())  # Number of graphs
            
            for graph_idx in range(k):
                # Read number of vertices and edges
                n, m = map(int, file.readline().split())  # Number of vertices and edges
                edges = []
                
                for _ in range(m):
                    edge_line = file.readline().strip()
                    if edge_line:  # Only process non-empty lines
                        u, v = map(int, edge_line.split())  # Read each edge
                        edges.append((u, v))
                
                if edges:
                    graphs.append((n, edges))
                else:
                    print(f"No edges found in graph {graph_idx + 1}")
    
    except Exception as e:
        print(f"Error reading file: {e}")
    
    return graphs

def main():
    input_file = "INPUT.TXT"  # Replace with your input file name
    graphs = read_graph_from_file(input_file)  # Read graphs from the file
    results = []

    # For each graph, check if it is bipartite
    for n, edges in graphs:
        result = is_bipartite(n, edges)
        results.append(result)

    # Output the results for each graph
    print(" ".join(map(str, results)))

if __name__ == "__main__":
    main()



-1 1
